In [8]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec
from eth_keys import keys
from eth_account.messages import encode_defunct
from eth_account import Account
from eth_utils import keccak
from pymerkle import InmemoryTree as MerkleTree
import os

# Utility function to check if a number is a power of two
def is_power_of_two(n):
    return n > 0 and (n & (n - 1)) == 0

# Function to load a private key from file
def load_private_key(file_path):
    with open(file_path, "rb") as f:
        private_key_bytes = f.read()
    return keys.PrivateKey(private_key_bytes)

# Function to compute SHA-256 hash of a message
def compute_sha256(message):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(message)   

    return keccak(message)  # Ethereum uses Keccak-256 for hashing the message


    return digest.finalize()

# Function to hash and sign a message
def sign_message(private_key, message):
    message_hash = compute_sha256(message)
    signature = private_key.sign_msg_hash(message_hash)
    return signature

# Function to verify a signature
def verify_signature(public_key, message, signature):
    message_hash = compute_sha256(message)
    return public_key.verify_msg_hash(message_hash, signature)

# Function to compute file hash (SHA-256)
def compute_file_hash(file_path):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    with open(file_path, 'rb') as f:
        for chunk in iter(lambda: f.read(4096), b""):
            digest.update(chunk)
    return digest.finalize().hex()

# Function to process data from a file using Merkle Tree and rounding
def process_file(file_path, skip=10, rounding=5):
    tree = MerkleTree(hash_type='sha256')
    roots = []
    
    with open(file_path, 'r') as f:
        for i, line in enumerate(f, start=1):
            if i % skip == 0:
                rounded_numbers = [round(float(num), rounding) for num in line.strip().split(',')]
                tree.append_entry(str(rounded_numbers).encode())
                if is_power_of_two(tree.get_size()):
                    roots.append(tree.get_state().hex())
    
    if not is_power_of_two(tree.get_size()):
        roots.append(tree.get_state().hex())
    
    return tree, roots

# Function to hash and sign an object (proof)
def sign_proof_object(private_key, proof_object):
    proof_str = str(proof_object).encode()
    proof_hash = compute_sha256(proof_str)
    signature = private_key.sign_msg_hash(proof_hash)
    return proof_hash.hex(), signature




In [9]:
# If you do not have one: Create a private key object

#private_key_bytes = os.urandom(32)
#private_key = keys.PrivateKey(private_key_bytes)
#with open("private_key.txt", "wb") as f:
#    f.write(private_key_bytes)


In [10]:
# Load private key from file
private_key = load_private_key("private_key.txt")
public_key = private_key.public_key

# Print keys and signature
#print(f"Private Key: {private_key}")
print(f"Public Key: {public_key}")

# Generate account from private key and print Ethereum address
account = Account.from_key(private_key)
print(f"Ethereum Address: {account.address}")
print("")

# Compute file hash
file_hash = compute_file_hash("input.txt")
code_hash = '000000001'


# Commit hashes into a pre-object
pre_object = {'code_hash': code_hash, 'file_hash': file_hash}
pre_hash, signature = sign_proof_object(private_key, pre_object)
    
# Verify pre-object signature
is_valid = public_key.verify_msg_hash(compute_sha256(str(pre_object).encode()), signature)

print(f"Committed pre hash: {pre_hash}")
print(f"Pre-object: {pre_object}")
print(f"Signature valid: {is_valid}")
print(f"Signature: {signature}")
print("")

# Run analysis and generate proof object
seed = 42
data_hash = '00000001'
_, roots = process_file("chain.txt", skip=10, rounding=5)
    
proof_object = {'roots': roots, 'data_hash': data_hash, 'seed': seed, 'pre_hash': pre_hash}
H_output, signature = sign_proof_object(private_key, proof_object)
    
# Verify proof object signature
is_valid = public_key.verify_msg_hash(compute_sha256(str(proof_object).encode()), signature)

print(f"Committed main hash: {H_output}")
print(f"Proof object: {proof_object}")
print(f"Signature valid: {is_valid}")
print(f"Signature: {signature}")

Private Key: 0x8122c7bbd23acf971e65a994744f6691b1b58bc5d0d7ff4bc5566e70f7005cf9
Public Key: 0xf47e6e2ee26af422643bed0c7f4eb96b9de00be9ab6a3e0ab9aca07338dd28f90c78d3c170c7fe2c06171ddf67747bc181625d1f3926cab8cbfc7794bab23b43
Ethereum Address: 0xa4903b76C11c730Fc97fa33087992c57a03e0b4d

Committed pre hash: fa2a1cd8684f118211889aeb7306f3444d7515b5ca409976e15c784a6e2cb170
Pre-object: {'code_hash': '000000001', 'file_hash': '218a6313c7d6ae484734a20395bd39efac1bb7fbd84f690afe94f77924649cae'}
Signature valid: True
Signature: 0x720890f2a5e67409d96edf6b153568b97d3dad10324992f4537134e8087dd8a16802954c31ae401bef3819fe71a25b20a674e6dfbe67a4ca983f3dbd7a6d1d5600

Committed main hash: 97223f37621f4f0b302cbf11aa149f1d69ea4c5544346ed1d3fad19d34d341f8
Proof object: {'roots': ['9aebface6387aeff2b3b4a9c0ccf94b617c41587a699d8b568ad7173806f28d9', '9ff5cf6b07c47ca91ea47969f88fdb4aec17760c74fecc89503fd4e25ae6d420', '169ed591e4712c61bdf73184d29ff53981c4832a2bc75bdbac5ca85de16f6776', '39f2cf5a5923eb790af160aef3c